Demo


참고 : https://github.com/deepinsight/insightface/tree/master/python-package

In [1]:
import cv2
import numpy as np
import insightface
from insightface.app import FaceAnalysis
from insightface.data import get_image as ins_get_image
from PIL import Image

app = FaceAnalysis(providers=['CUDAExecutionProvider', 'CPUExecutionProvider']) # 모델 이름을 인자로 줘서 원하는 모델 사용 가능
app.prepare(ctx_id=0, det_size=(640, 640)) # det_size 는 face detection 후에 crop된 얼굴 크기를 얼마나의 크기로 resize할 것인지를 의미한다.

Applied providers: ['CUDAExecutionProvider', 'CPUExecutionProvider'], with options: {'CUDAExecutionProvider': {'do_copy_in_default_stream': '1', 'cudnn_conv_algo_search': 'EXHAUSTIVE', 'device_id': '0', 'gpu_external_alloc': '0', 'enable_cuda_graph': '0', 'gpu_mem_limit': '18446744073709551615', 'gpu_external_free': '0', 'gpu_external_empty_cache': '0', 'arena_extend_strategy': 'kNextPowerOfTwo', 'cudnn_conv_use_max_workspace': '0', 'cudnn_conv1d_pad_to_nc1d': '0'}, 'CPUExecutionProvider': {}}
find model: C:\Users\ParkMinSu/.insightface\models\buffalo_l\1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CUDAExecutionProvider', 'CPUExecutionProvider'], with options: {'CUDAExecutionProvider': {'do_copy_in_default_stream': '1', 'cudnn_conv_algo_search': 'EXHAUSTIVE', 'device_id': '0', 'gpu_external_alloc': '0', 'enable_cuda_graph': '0', 'gpu_mem_limit': '18446744073709551615', 'gpu_external_free': '0', 'gpu_external_empty_cache': '0', 'arena_extend_strategy': 'k

In [2]:
def get_embed(path): # Input : rgb 이미지의 경로
                     # Output : 이미지에 있는 얼굴들의 임베딩
    img = np.asarray(Image.open(path))
    img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR) # 이미지를 BGR 형식으로 바꿔 주어야 한다.
    faces = app.get(img)
    embeddings = []
    for tmp in faces:
        embeddings.append(tmp["embedding"])
    
    return embeddings

In [3]:
def dist(embed_1, embed_2): # Input : 두 그림의 embedding
                            # Output : 두 그림 사이의 거리
                            # 거리를 측정하는 방식들을 여러 가지로 구현해 보면서 성능 향상을 노려볼 수 있을 것 같다.
    return np.linalg.norm(embed_1 - embed_2)

In [4]:
def target_in_image(path_image, target_embedding, threshold = 20): # Input : rgb 이미지의 경로, 사진 속에 있는지 확인하고 싶은 사람(타겟)의 임베딩, threshold
                                                              # Output : 이미지에 해당 사람(타겟)이 있는지 true/false
                                                              # 적절한 threshold 값을 선택하여 성능 향상을 노려볼 수 있을 것 같다.
    embeddings = get_embed(path_image)
    for embedding in embeddings:
        if(dist(embedding, target_embedding) < threshold):
            return True # target exist in image
    return False # target is not found in image

In [5]:
image_folder_path = "images/"
path_list = ["img1.jpg", "img2.png", "img3.jfif", "img4.jpg"]

target_image = "target.jpg" # 유재석의 사진
target_embed = get_embed(image_folder_path + target_image)[0] # 유재석의 임베딩

c:\Users\ParkMinSu\AppData\Local\Programs\Python\Python37\lib\site-packages\insightface\utils\transform.py:68: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  P = np.linalg.lstsq(X_homo, Y)[0].T # Affine matrix. 3 x 4


In [6]:
for path in path_list:
    print(target_in_image(image_folder_path + path, target_embed))
    

False
True
True
False
